In [ ]:
# Things left to do
# Need to map to optimizely_dashbaord_results to figure out which

In [4]:
import import_ipynb
import os

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import date

import scipy.stats as stats
from scipy.stats import chi2_contingency
import seaborn as sns

In [8]:
import import_data

importing Jupyter notebook from import_data.ipynb
Connected to snowflake. Now read in query = open('/Folder Directory/sql_query_of_data.sql','r').read()


### Input Variables

In [251]:
# Input variables
experiment_id = '20053754433'
conv_event = 'invitationSent'

temp_experiment_filter = 'AND (EXPERIMENT_ID LIKE \'%'+experiment_id+'%\')'

print(temp_experiment_filter,'\n',conv_event)

AND (EXPERIMENT_ID LIKE '%20053754433%') 
 invitationSent


### Import data

#### Import Dashboard Results

In [9]:
# query =\
# """
# SELECT 		EXPERIMENT_ID AS "experiment_id"
# 		,EXPERIMENT_NAME AS "experiment_name"
# 		,VARIATION_ID AS "variation_id"
# 		,VARIATION_NAME AS "variation_name"
# 		,MAX(IS_BASELINE_VARIATION) AS "is_baseline_variation"
# --			,MIN(START_TIME_UTC),MAX(END_TIME_UTC),COUNT(DISTINCT VARIATION_ID),MAX(VARIATION_ID),MIN(VARIATION_ID)
# FROM 		PUBLIC.OPTIMIZELY_DASHBOARD_RESULTS 
# WHERE 		EXPERIMENT_NAME LIKE 'growth-%'
# GROUP BY 	1,2,3,4
# HAVING 		DATEDIFF(DD,MAX(END_TIME_UTC),CURRENT_DATE) BETWEEN 0 AND 1
# ORDER BY 	MAX(END_TIME_UTC) DESC, EXPERIMENT_NAME DESC
# """
# db_results = import_data.query_to_df(query)
# db_results

#### Import IMPR_EVENTS

In [10]:
# query =\

#### Import ENR_DECI

In [11]:
# Test enr_deci query for now
query =\
"""
SELECT	 VISITOR_ID AS "visitor_id"
		,EXPERIMENT_ID AS "experiment_id"
		,VARIATION_ID AS "variation_id"
		,GUID AS "guid"
		,ENVIRONMENT AS "environment"
		,TIMESTAMP AS "timestamp"
FROM PUBLIC.OPTIMIZELY_ENR_DECI 
WHERE ENVIRONMENT='production'
"""\
+temp_experiment_filter
# WHERE CREATED_AT >= """+'\''+dt_from.strftime("%Y-%m-%d")+'\''

enr_deci = import_data.query_to_df(query)


SELECT	 VISITOR_ID AS "visitor_id"
		,EXPERIMENT_ID AS "experiment_id"
		,VARIATION_ID AS "variation_id"
		,GUID AS "guid"
		,ENVIRONMENT AS "environment"
		,TIMESTAMP AS "timestamp"
FROM PUBLIC.OPTIMIZELY_ENR_DECI 
WHERE ENVIRONMENT='production'
AND (EXPERIMENT_ID LIKE '%20053754433%')


In [12]:
enr_deci.head()

,visitor_id,experiment_id,variation_id,guid,environment,timestamp
0,5f76895e-2979-4cfc-82a7-5897028f454a,20053754433,20057713467,27f9b402-6f8d-25eb-8070-38c4c969916c,production,2021-03-19 16:56:22.871
1,74720145-6055-8c7f-804b-7af3c89044ae,20053754433,20041984016,68d2e698-2058-24fd-54da-a698e5cde600,production,2021-03-19 00:04:17.877
2,68419b6f-345f-40ee-9df7-d04a7352431d,20053754433,20041984016,68419b6f-345f-40ee-9df7-d04a7352431d,production,2021-03-19 18:00:45.946
3,e6cb650f-ab06-48d6-ba1a-6a8d3d6c00f8,20053754433,20057713467,e6cb650f-ab06-48d6-ba1a-6a8d3d6c00f8,production,2021-03-19 22:59:25.832
4,7268b926-985d-c08b-be54-276c32d7cbd1,20053754433,20041984016,e433d1f4-46f9-4455-a9ff-3d1aa957fad0,production,2021-03-19 21:17:23.496


#### Import ENR_CONV

In [13]:
# Test enr_conv query for now
query =\
"""
SELECT	 VISITOR_ID AS "visitor_id"
        ,EVENT_NAME AS "event_name"
		,EXPERIMENT_ID AS "experiment_id"
		,VARIATION_ID AS "variation_id"
		,GUID AS "guid"
		,TIMESTAMP AS "timestamp"
FROM PUBLIC.OPTIMIZELY_ENR_CONV
WHERE ENVIRONMENT='production'
"""\
+temp_experiment_filter

enr_conv=import_data.query_to_df(query)


SELECT	 VISITOR_ID AS "visitor_id"
        ,EVENT_NAME AS "event_name"
		,EXPERIMENT_ID AS "experiment_id"
		,VARIATION_ID AS "variation_id"
		,GUID AS "guid"
		,TIMESTAMP AS "timestamp"
FROM PUBLIC.OPTIMIZELY_ENR_CONV
WHERE ENVIRONMENT='production'
AND (EXPERIMENT_ID LIKE '%20053754433%')


In [14]:
enr_conv.head()

,visitor_id,event_name,experiment_id,variation_id,guid,timestamp
0,94955520-3170-4cfc-8dec-3415c219dd8e,Recommendations Badge Filter Changed,"19978184566, 20053754433, 19947312075","19971631161, 20041984016, 19924881859",94955520-3170-4cfc-8dec-3415c219dd8e,2021-03-16 17:52:22.972
1,de326e3a-7f32-4e5f-9df7-656e4d01b33e,Recommendations Viewed,"19978184566, 20053754433, 19947312075, 198951...","19980937118, 20041984016, 19945381330, 199074...",de326e3a-7f32-4e5f-9df7-656e4d01b33e,2021-03-16 19:03:24.315
2,d92cbeb2-1630-4ae5-ae99-74d948bc58db,Recommendations Viewed--homepage,"19978184566, 20053754433, 19947312075, 198951...","19971631161, 20057713467, 19924881859, 199074...",d92cbeb2-1630-4ae5-ae99-74d948bc58db,2021-03-16 18:57:33.731
3,e036ddd9-648c-4c6d-85c4-2df39d4cfec5,Form Viewed--homepage,"19978184566, 20053754433, 19847400444, 199473...","19971631161, 20057713467, 19732568477, 199248...",e036ddd9-648c-4c6d-85c4-2df39d4cfec5,2021-03-16 20:48:47.048
4,ceda5767-c091-4ee5-9595-c7d223e08605,Form Viewed,20053754433,20041984016,ceda5767-c091-4ee5-9595-c7d223e08605,2021-03-16 19:59:37.589


#### Import DW_EVENTS

In [75]:
# Set platform 
platform = 'web'

web_query =\
"""
SELECT DISTINCT GUID AS "guid",CLIENT_PUBLISH_TYPE AS "client_publish_type",CREATED_AT AS "created_at",'web' AS "platform"
FROM PUBLIC.JOB_FLOW_EVENTS
"""

native_query =\
"""
SELECT DISTINCT GUID AS "guid",CLIENT_PUBLISH_TYPE AS "client_publish_type",CREATED_AT AS "created_at",'native' AS "platform"
FROM PUBLIC.REACT_NATIVE_EVENTS
"""

if platform == 'web':
    dw_query = web_query
elif platform == 'native':
    dw_query = native_query
elif platform == 'both':
    dw_query = 'SELECT * FROM ('+web_query+'UNION'+native_query+')'
    
print(dw_query)


SELECT DISTINCT GUID AS "guid",CLIENT_PUBLISH_TYPE AS "client_publish_type",CREATED_AT AS "created_at",'web' AS "platform"
FROM PUBLIC.JOB_FLOW_EVENTS



In [76]:
first_timestamp = enr_deci.timestamp.min().strftime('%Y-%m-%d')
first_timestamp

query = dw_query+\
' WHERE "created_at" >= \''+first_timestamp+'\''
print(query)


SELECT DISTINCT GUID AS "guid",CLIENT_PUBLISH_TYPE AS "client_publish_type",CREATED_AT AS "created_at",'web' AS "platform"
FROM PUBLIC.JOB_FLOW_EVENTS
 WHERE "created_at" >= '2021-03-16'


In [77]:
dw_events=import_data.query_to_df(query)
dw_events.head()


SELECT DISTINCT GUID AS "guid",CLIENT_PUBLISH_TYPE AS "client_publish_type",CREATED_AT AS "created_at",'web' AS "platform"
FROM PUBLIC.JOB_FLOW_EVENTS
 WHERE "created_at" >= '2021-03-16'


In [78]:
dw_events.head()

,guid,client_publish_type,created_at,platform
0,e7ada8cd-6547-4495-809a-ef9de1943eb5,Recommendations Date Filter Changed,2021-03-16 00:02:06,web
1,d3e0b7e5-0d65-400d-aeeb-66624cf96941,Recommendations Viewed,2021-03-16 00:00:50,web
2,afa49b4c-4a0b-48c1-a295-df4de28f4a4e,Tasker Schedule Viewed,2021-03-16 00:00:37,web
3,7f8e4ebd-381c-4d7d-a117-3f2e0e717aeb,Confirmation Signup Viewed,2021-03-16 00:01:38,web
4,e8d8b097-4353-459c-8616-38b8dddb2d20,Location_Service_Page_Viewed,2021-03-16 00:00:22,web


### Data wrangling

In [233]:
print(dw_events.shape
,enr_deci.shape
,enr_conv.shape)

(12180546, 4) (696898, 6) (7981065, 6)


In [234]:
opt_summary = enr_deci.groupby(['experiment_id','variation_id']).agg({'timestamp':['min','max'],'guid':'nunique'}).reset_index()
opt_summary.columns=['experiment_id','variation_id','first_timestamp','last_timestamp','guid_count']
opt_summary

,experiment_id,variation_id,first_timestamp,last_timestamp,guid_count
0,20053754433,20041984016,2021-03-16 17:11:46.770,2021-04-20 23:59:41.788,331392
1,20053754433,20057713467,2021-03-16 17:11:46.861,2021-04-20 23:59:37.882,332095


In [235]:
# Add AB test duration col
opt_summary['duration'] = opt_summary.apply(lambda x: (x['last_timestamp'] - x['first_timestamp']).days,axis=1)
opt_summary

,experiment_id,variation_id,first_timestamp,last_timestamp,guid_count,duration
0,20053754433,20041984016,2021-03-16 17:11:46.770,2021-04-20 23:59:41.788,331392,35
1,20053754433,20057713467,2021-03-16 17:11:46.861,2021-04-20 23:59:37.882,332095,35


In [236]:
def count_conversions(exp_id,var_id,evt_name):
    conv_in_exp_var = enr_conv[\
                           (enr_conv['experiment_id'].str.find(exp_id)!=-1)\
                           & (enr_conv['variation_id'].str.find(var_id)!=-1)\
                            & (enr_conv['event_name'] == evt_name)\
                          ]
    conv_by_event = conv_in_exp_var.groupby('event_name').agg({'guid':'nunique'}).reset_index()
    return(conv_by_event.guid)

In [237]:
conv_by_event = count_conversions('20053754433','20041984016','invitationSent')
conv_by_event

0    44742
Name: guid, dtype: int64

In [238]:
# Add conversion count + conversion rate columns
opt_summary['conv_count'] = opt_summary.apply(lambda x: count_conversions(x['experiment_id'],x['variation_id'],conv_event),axis=1)
opt_summary['conv_rate'] = opt_summary.conv_count / opt_summary.guid_count
opt_summary

,experiment_id,variation_id,first_timestamp,last_timestamp,guid_count,duration,conv_count,conv_rate
0,20053754433,20041984016,2021-03-16 17:11:46.770,2021-04-20 23:59:41.788,331392,35,44742,0.135012
1,20053754433,20057713467,2021-03-16 17:11:46.861,2021-04-20 23:59:37.882,332095,35,44913,0.135241


In [224]:
def get_dw_info(exp_id,var_id,evt_name): # Pass in 3 vars from summary table
    guids = enr_conv[\
                       (enr_conv['experiment_id'].str.find(exp_id)!=-1)\
                       & (enr_conv['variation_id'].str.find(var_id)!=-1)\
                      ]
    guids = guids.guid.unique() # num guids in the experiment+variation
    
    dw = dw_events[(dw_events.created_at>='2021-03-16') & (dw_events.created_at<='2021-04-20')\
                   & (dw_events.guid.isin(guids))] # dw events from guids in the experiment+variation
    
    guid_ct = dw.guid.nunique()
    conv_ct = dw[dw.client_publish_type==evt_name]
    conv_ct = conv_ct.guid.nunique()
    first_date = dw.created_at.min()
    last_date = dw.created_at.max()
    
    return guid_ct,conv_ct,first_date,last_date # Return event info from datawarehouse

In [239]:
# Create df for datawarehouse info from function
dw_info = opt_summary.apply(lambda x : get_dw_info(x['experiment_id'],x['variation_id'],conv_event),axis=1)
dw_info = pd.DataFrame(dw_info.values.tolist(),columns=['dw_guid_count','dw_conv_count','dw_first_date','dw_last_date'])

In [247]:
summary = pd.concat([opt_summary,dw_info],axis=1)
summary['dw_conv_rate'] = summary.dw_conv_count / summary.dw_guid_count
summary

,experiment_id,variation_id,first_timestamp,last_timestamp,guid_count,duration,conv_count,conv_rate,dw_guid_count,dw_conv_count,dw_first_date,dw_last_date,dw_conv_rate
0,20053754433,20041984016,2021-03-16 17:11:46.770,2021-04-20 23:59:41.788,331392,35,44742,0.135012,216464,51042,2021-03-16 00:00:01,2021-04-20 00:00:00,0.235799
1,20053754433,20057713467,2021-03-16 17:11:46.861,2021-04-20 23:59:37.882,332095,35,44913,0.135241,217350,51310,2021-03-16 00:00:02,2021-04-19 23:59:59,0.236071


In [ ]:
# Ideal format for the tests
# User ID | Variant | Convert [0,1]

### Data Manipulation

In [ ]:
# Check for null values
print(len(data[data.search_bfs.isna()]))
print(len(data[data.bfs.isna()]))
print(len(data[data.booked.isna()]))

### EDA

In [ ]:
# Split into control vs. search update variants
control=data[data.variation=='control']
search_update=data[data.variation=='search_update']

print(len(control),'+',len(search_update),'=',len(control)+len(search_update))
print(len(data))

### Run Mann-Whitney Test

In [ ]:
# mann_whitney function that inputs the control + variant conversion columns
def mann_whitney(control,variant,conv):
    u_statistic, p_value = stats.mannwhitneyu(control[conv],variant[conv])
    print("u_statistic: \t",u_statistic,"\np_value: \t",p_value)
    if p_value<0.05:
        print("\n Mann-Whitney test is significant for ",conv)

In [ ]:
mann_whitney(control,search_update,'search_bfs')

In [ ]:
mann_whitney(control,search_update,'bfs')

In [ ]:
# mann_whitney(control,search_update,'booked')

### Run chi-square test

In [ ]:
def chi2_test(variations,conv):    
    contingency = pd.crosstab(variations, data[conv]) 
    display(contingency)
    
    contingency_pct = pd.crosstab(variations, data[conv], normalize='index')
    display(contingency_pct)
    
#     plt.figure(figsize=(4,2)) 
#     sns.heatmap(contingency, annot=True, cmap="YlGnBu")
    
    c, p, dof, expected = chi2_contingency(contingency)
    print('\np-value:\t',p)
    if p<0.05:
        print("\n Chi-square test is significant for",conv)

In [ ]:
chi2_test(data['variation'],'search_bfs')

In [ ]:
chi2_test(data['variation'],'bfs')

In [ ]:
# chi2_test(data['variation'],'booked')

### Run Fishers Test

In [ ]:
def fisher_exact(variations,conv):    
    contingency = pd.crosstab(variations, data[conv]) 
    display(contingency)
    
    oddsratio, p_value = stats.fisher_exact(contingency)

    print('\np-value:\t',p_value)
    if p_value<0.05:
        print("\n Fisher exact test is significant for",conv)

In [ ]:
fisher_exact(data['variation'],'search_bfs')

In [ ]:
fisher_exact(data['variation'],'bfs')

In [ ]:
fisher_exact(data['variation'],'booked')

### Calculate lift across search BFS, overall BFS, and booking rates

In [ ]:
round(3.23432,2)

In [ ]:
# calculate_lift function that inputs the control + variant dfs
def calculate_lift(control,variant,conv):
    control_conv=100*(control[conv].sum()/control['guid'].count())
    print('CONTROL')
    print('total visitors:',control['guid'].count())
    print('unique conversions:',control[conv].sum())
    print('control ',conv,' rate:','%.2f'%control_conv)

    print('\nVARIANT')
    print('total visitors:',variant['guid'].count())
    print('unique conversions:',variant[conv].sum())
    variant_conv=100*(variant[conv].sum()/variant['guid'].count())
    print('variant ',conv,' rate:','%.2f'%variant_conv)
    
    print('\nLIFT - CONTROL VS. VARIANT')
    lift_magnitude = variant_conv-control_conv
    lift_pct = 100*(lift_magnitude/control_conv)
#     print('Lift (magnitude) in',conv,':','%.2f'%lift_magnitude)
    print('Lift (percent) in',conv,':','%.2f'%lift_pct,'percent','(','%.2f'%control_conv,'vs.','%.2f'%variant_conv,')')

In [ ]:
calculate_lift(control,search_update,'search_bfs')

In [ ]:
calculate_lift(control,search_update,'bfs')

In [ ]:
calculate_lift(control,search_update,'booked')